# GeoTIFF Downloads

Get landcover layers (actually dynamic layers), because these have our tags

In [57]:
import urllib
import json
import requests
import sys, traceback
from http.client import responses
import os

In [58]:
# this needs to be reset from time to time based on scraping params from browser
csrftoken = 'K0lvdkPm6o3kZScD0mVP9xdOl5souWxc9iFEMKHTQGzsMLi4uxNSJ4QA4DY2jAA8'
sessionid = 'wy8b3yrwcdr8tlj3d9puzo45q017yi1c'
cookie_str = 'csrftoken=' + csrftoken + ';' 
cookie_str += 'sessionid='+ sessionid
headers = {'X-CSRFToken': csrftoken,'cookie': cookie_str}

In [59]:
landcover_url = 'http://greatbasin.geodesigntech.com/api/landcoverlayer'

In [60]:
def get_landcover_dict():
    result = requests.get(landcover_url,headers=headers)
    return(result.json())

In [61]:
lcd = get_landcover_dict()

In [62]:
# this has a count
lcd_count = lcd['count']
print('We have got {} layers'.format(lcd_count))

We have got 276 layers


In [63]:
lct = None
lct = lcd['results'][0]

In [64]:
lct

{'end_date': '2017-03-02',
 'filename': 'CBR_IV_C_NS_ANNUALGRASSES_COMBINED.tif',
 'formula': '',
 'id': 401,
 'layers': '',
 'legend': [{'code': '',
   'color': '#3988e8',
   'expression': '1',
   'name': '1 - < 5% Risk'},
  {'code': '',
   'color': '#80ffff',
   'expression': '2',
   'name': '2 - 5-15% Risk'},
  {'code': '',
   'color': '#ffff80',
   'expression': '3',
   'name': '3 - 15-25% Risk'},
  {'code': '', 'color': '#ff8000', 'expression': '4', 'name': '4 - 25-45%'},
  {'code': '',
   'color': '#ff0000',
   'expression': '5',
   'name': '5 - > 45% Risk'}],
 'legend_id': 7,
 'legend_name': 'Mask',
 'name': 'Annual Grasslands',
 'raster': 41,
 'start_date': '2017-03-02',
 'tags': ['Taxa: Plants', 'Year: 1990', 'resource']}

### Load Landcover Layers metadata into dataframe

In [65]:
# build a pandas dataframe for easy access and counting
import pandas as pd

lcd_df = pd.DataFrame(columns=['layerid','name','type','scenario','month','season','year','legend_name','legend','formula','layers','variable'])

In [66]:
rows_list = []
for layer in lcd['results']:
    legend_name = layer['legend_name']
    legend = layer['legend'] # the dictionary of the legend
    tags_list = layer['tags']
    num_tags = len(tags_list)
    #print('Number of tags is {}'.format(num_tags))
    #print("Tags list is: {}".format(tags_list))
    for t in tags_list:
        if ':' in t:
            tag_type = 'pair'
            tag_group = t.split(':')[0]
            tag_value = t.split(':')[1].strip() # get rid of extra starting space
        else:
            tag_type = 'single'
            tag_group = ''
            tag_value = t.strip() # get rid of extra starting space
        layerDict = {} # make sure we have a dict to update
        # we have these for all layers
        layerDict.update({'layerid': layer['id'], 'name': layer['name']})
        layerDict.update({'legend_name': legend_name})
        layerDict.update({'legend': legend})
        layerDict.update({'formula': layer['formula']})
        layerDict.update({'layers': layer['layers']})
        # in case where landscape is a pointer to a single raster, then layers & formula can be empty
        if len(layer['formula']) == 0:
            # make a trivial formula and layer dictionary
            identity_formula = 'x'.format(layer['raster'])
            identity_layers = 'x = {}'.format(layer['raster'])
            layerDict.update({'formula': identity_formula, 'layers': identity_layers})
        if tag_group == 'Variable':
            layerDict.update({'variable': tag_value})
        if tag_value == 'resource':
            #print('Resource: ' + layer['name'], tags)
            layerDict.update({'type': 'resource'})
        if tag_value == 'driver':
            #print('Driver: ' + layer['name'], tags)
            layerDict.update({'type': 'driver'})
        if tag_group == 'Scenario':
            #print('  found a scenario tag')
            layerDict.update({'scenario': tag_value})
        if tag_group == 'Year':
            #print('  found a year tag')
            layerDict.update({'year': tag_value})
        if tag_group == 'Season':
            #print('  found a season tag')
            layerDict.update({'season': tag_value})
        #print('debug: layer update dictionary: {}'.format(layerDict))
        rows_list.append(layerDict)

# add big rows list only once at end for efficiency (relative to row-wise append)
lcd_df = pd.DataFrame(rows_list)               


In [67]:
lcd_df.shape

(296, 11)

In [68]:
lcd_df.head()

,formula,layerid,layers,legend,legend_name,name,scenario,season,type,variable,year
0,x,401,x = 41,"[{'code': '', 'expression': '1', 'name': '1 - ...",Mask,Annual Grasslands,NaN,NaN,NaN,NaN,NaN
1,x,401,x = 41,"[{'code': '', 'expression': '1', 'name': '1 - ...",Mask,Annual Grasslands,NaN,NaN,NaN,NaN,1990
2,x,401,x = 41,"[{'code': '', 'expression': '1', 'name': '1 - ...",Mask,Annual Grasslands,NaN,NaN,resource,NaN,NaN
3,x,414,x = 733,"[{'code': '5', 'expression': '1', 'name': 'Agr...",Desert Bighorn Sheep Habitat Types,California Desert Bighorn Sheep Distribution 2016,NaN,NaN,resource,NaN,NaN
4,x,414,x = 733,"[{'code': '5', 'expression': '1', 'name': 'Agr...",Desert Bighorn Sheep Habitat Types,California Desert Bighorn Sheep Distribution 2016,NaN,NaN,NaN,NaN,NaN


In [ ]:
variables = lcd_df['variable'].unique()
variables

In [27]:
scenarios = lcd_df[lcd_df['scenario'].notna()]
scenarios

,formula,layerid,layers,legend,legend_name,name,scenario,season,type,variable,year
12,(A1b2050 - A2_2010) <> 0,428,"A1b2050=773,A2_2010=779","[{'code': '', 'expression': '(x > 0) or (x < 0...",None,Fore-Sce A1b Land Cover Change 2010-2050,IPCC A1b,NaN,NaN,NaN,NaN
17,x,429,x = 776,"[{'code': '', 'expression': '1', 'name': 'Deve...",Developed,Fore-Sce A1b Urbanization 2010-2040,IPCC A1b,NaN,NaN,NaN,NaN
23,((ForeSce2050 - ForeSce2010) <> 0) * GB,425,"ForeSce2050=782,ForeSce2010=779,GB=647","[{'code': '', 'expression': '(x > 0) or (x < 0...",None,Fore-Sce A2 Land Cover Change 2050,IPCC RCP 8.5,NaN,NaN,NaN,NaN
29,x,392,x = 670,"[{'code': '', 'expression': '13', 'name': 'Agr...",USGS Fore-sce Scenarios,Fore-sce Scenario A1B 2050,IPCC A1B,NaN,NaN,NaN,NaN
35,x,390,x = 669,"[{'code': '', 'expression': '13', 'name': 'Agr...",USGS Fore-sce Scenarios,Fore-sce Scenario A2 2050,IPCC A2,NaN,NaN,NaN,NaN
42,x,391,x = 671,"[{'code': '', 'expression': '13', 'name': 'Agr...",USGS Fore-sce Scenarios,Fore-sce Scenario B1 2050,IPCC B1,NaN,NaN,NaN,NaN
46,x,407,x = 659,"[{'code': '', 'expression': 'x > 0', 'name': '...",Predicted Habitat Presence,Great Basin Pinyon Juniper Woodland 2010,Baseline,NaN,NaN,NaN,NaN
51,x,402,x = 719,"[{'code': '', 'expression': 'x > 0', 'name': '...",Bioclimate Suitability Level 0-6,Greater Sage-grouse,Baseline,NaN,NaN,NaN,NaN
53,x,411,x = 699,"[{'code': '1', 'expression': '1', 'name': '1 G...",Bioclimate Suitability Level 0-6,Greater Sage Grouse 2050 Bioclimate,IPCC RCP 8.5,NaN,NaN,NaN,NaN
56,cbr * (mar + apr + may),375,"mar=643,apr=481,may=447,cbr=647","[{'code': 'A', 'expression': '(x<(25.4*3))', '...",None,Historical 1990 Q1 Spring Total Precipitation,Baseline,NaN,NaN,NaN,NaN


In [15]:
print('We have {} scenario grids'.format(len(scenarios)))

In [16]:
lcd_df['scenario'].unique()

array([nan, 'IPCC A1b', 'IPCC RCP 8.5', 'IPCC A1B', 'IPCC A2', 'IPCC B1',
       'Baseline', 'Baselinen', 'IPCC RCP4.5', 'IPCC RCP8.5'],
      dtype=object)

In [17]:
lcd_df['year'].unique()

array([nan, '1990', '2050', '2040', '2012', '2010', '2011', '2060',
       '2030'], dtype=object)

In [18]:
lcd_df['season'].unique()

array([nan, 'annual', 'spring', 'summer', 'fall', 'winter'], dtype=object)

In [19]:
def scenarioGrids(scenario):
    return(lcd_df[lcd_df['scenario'] == scenario])

In [20]:
scenarioGrids('Baseline').head(3)

,formula,layerid,layers,legend,legend_name,name,scenario,season,type,variable,year
46,x,407,x = 659,"[{'expression': 'x > 0', 'code': '', 'name': '...",Predicted Habitat Presence,Great Basin Pinyon Juniper Woodland 2010,Baseline,NaN,NaN,NaN,NaN
51,x,402,x = 719,"[{'expression': 'x > 0', 'code': '', 'name': '...",Bioclimate Suitability Level 0-6,Greater Sage-grouse,Baseline,NaN,NaN,NaN,NaN
56,cbr * (mar + apr + may),375,"mar=643,apr=481,may=447,cbr=647","[{'expression': '(x<(25.4*3))', 'code': 'A', '...",None,Historical 1990 Q1 Spring Total Precipitation,Baseline,NaN,NaN,NaN,NaN


In [21]:
def get_landcover_layer_dict(id):
    raster_layer_url = 'http://greatbasin.geodesigntech.com/api/landcover/{}?format=json'
    url = raster_layer_url.format(id)
    result = requests.get(url,headers=headers)
    if result.status_code == requests.codes.ok:
        print('Found landcover layer of specified ID')
        return(result.json())
    else:
        print('Did not find landcover layer of specified ID')
        return(None)

In [ ]:
get_landcover_layer_dict(443)

In [ ]:
def get_raster_layer_dict(id):
    raster_layer_url = 'http://greatbasin.geodesigntech.com/api/rasterlayer/{}?format=json'
    url = raster_layer_url.format(id)
    result = requests.get(url,headers=headers)
    return(result.json())

In [ ]:
get_raster_layer_dict(41)

### Legends and Posting Full Landcover Layer Definitions

In [ ]:
driver_grid_id = 402 # bioclim

In [ ]:
# df_test['Btime'].iloc[0]
driver_legend = scenarios[scenarios.layerid == driver_grid_id]['legend'].iloc[0]
driver_legend

In [ ]:
lct['legend'] = str(driver_legend).replace('[','').replace(']','')
# to do: carry legend_id and name over

In [ ]:
lct

In [ ]:
lct['legend_name'] = 'Fore-Sce Land Cover Classes'
lct['name'] = 'test of landcover post'
#lct.pop('raster')
# 'tags': ['Taxa: Plants', 'Year: 1990', 'resource']}
lct['tags'] = ['exposure', 'Year: 2060']

In [ ]:
# drop the id for posting...
lct.pop('id')

In [ ]:
def post_landcover_dict():
    try:
        result = requests.post(landcover_url,headers=headers,data=lct)
        result.raise_for_status()
        # shouldn't just be requests.codes.ok, should actually be 'created'
        if result.status_code == requests.codes.created:
            print('Resource "{}" created on server'.format(lct['name']))
            return(result.json())
        else:
            print('Server responded negatively to post request, result code: {}'.format(result.status_code))
            return(None)
    except:
        print('Problem posting landcover layer')

In [ ]:
new_lc = post_landcover_dict()
new_lc

In [ ]:
get_landcover_layer_dict(new_lc['id'])

In [ ]:
# try patching existing resources, for example adding tags systematically?

In [ ]:
import os
def patch_landcover_dict(layer_id, patch_dict):
    patch_url = os.path.join(landcover_url, str(layer_id))
    try:
        result = requests.patch(patch_url,headers=headers,json=patch_dict)
        result.raise_for_status()
        # shouldn't just be requests.codes.ok, should actually be 'created'
        if result.status_code == requests.codes.ok:
            print('  Land cover layer id "{}" patched on server with result code {}'.format(layer_id, result.status_code))
            return(result.json())
        else:
            print('  Server responded negatively to patch request, result code: {}'.format(result.status_code))
            return(None)
    except:
        print('  Problem patching landcover layer')

### Generating Tags by Parsing File Names

In [ ]:
for layer in lcd_df.itertuples():
    # tag based on climate change variable in name
    if 'temperature' in layer.name.lower():
        print('Found a temperature grid, updating tags')
        patch_dict = {'tags': ['Variable: temperature']}
        patch_landcover_dict(layer.layerid, patch_dict)
    if 'precipitation' in layer.name.lower():
        print('Found a precipitation grid, updating tags')
        patch_dict = {'tags': ['Variable: precipitation']}
        patch_landcover_dict(layer.layerid, patch_dict)
    if 'bioclimate' in layer.name.lower():
        print('Found a bioclimate grid, updating tags')
        patch_dict = {'tags': ['Variable: bioclimate']}
        patch_landcover_dict(layer.layerid, patch_dict)

    # to do: make explicit date ranges and find dates in names
    if '2030' in layer.name:
        patch_dict = {'tags': ['Year: 2030', 'Epoch: midterm-future']}
        patch_landcover_dict(layer.layerid, patch_dict)

    if '2040' in layer.name:
        patch_dict = {'tags': ['Year: 2040', 'Epoch: midterm-future']}
        patch_landcover_dict(layer.layerid, patch_dict)

    if '2060' in layer.name:
        patch_dict = {'tags': ['Year: 2060', 'Epoch: longterm-future']}
        patch_landcover_dict(layer.layerid, patch_dict)

        
    scenarioInName = ('RCP' in layer.name) or ('Historical' in layer.name)
    isScenario = str(layer.scenario) != 'nan'
    if scenarioInName and not isScenario:
        print("Checking layer '{}'".format(layer.name))
        # 'tags': ['Taxa: Plants', 'Year: 1990', 'resource']}
        print('  Updating {} tags'.format(layer.name))
        if '4.5' in layer.name or '45' in layer.name:
            patch_dict = {'tags': ['scenario: IPCC RCP4.5', 'driver']}
            patch_landcover_dict(layer.layerid, patch_dict)
        elif '8.5' in layer.name or '85' in layer.name:
            patch_dict = {'tags': ['scenario: IPCC RCP8.5', 'driver']}
            patch_landcover_dict(layer.layerid, patch_dict)
        elif 'Historical' in layer.name or '1990' in layer.name:
            patch_dict = {'tags': ['scenario: Baseline', 'driver']}
            patch_landcover_dict(layer.layerid, patch_dict)
        else:
            print('hmmm...confused')

In [ ]:
# required are
# name, raster, start, end, formula

In [ ]:
lcd_df.head()

### Legend

In [ ]:
# try to get legend/entries and their expressions

In [ ]:
legend_name = 'Residential Housing Density'

In [ ]:
legend_json = '[{"color": "#ffff99", "code": "", "expression": "( x < 0.1)", "name": "0 - 0.1"}, {"color": "#ffff00", "code": "", "expression": "(x >= 0.1) and (x < 1)", "name": "0.1 - 1.0"}, {"color": "#ff9900", "code": "", "expression": "(x >= 1) and (x < 2)", "name": "1 - 2"}, {"color": "#ff6600", "code": "", "expression": "(x >= 2) and ( x < 5)", "name": "2 - 5"}, {"color": "#ff6600", "code": "", "expression": "(x >= 5) and (x < 10)", "name": "5 - 10"}, {"color": "#993300", "code": "", "expression": "x > 10", "name": "> 10"}]'

In [ ]:
# example manually pulled legend entry expression
legend_class = '(x >= 0.1) and (x < 1)'

In [ ]:
formula_test = '(development>0) * wabbits' # no classes here, just reflecting habitat masked inputs

In [ ]:
# habitat for humanity (theobold's predicted 2030 residentail density in du/ac)
layer_test = 'development=684,wabbits=724'

### Extract Basic Parameters Reverse-engineering working URL

In [22]:
url = 'http://greatbasin.geodesigntech.com/raster/export?bbox=-130.7373046875,33.063924198120645,-103.447265625,43.723474896114794&zoom=6&layers=habitat=41,mar=643,apr=481,may=447,cbr=647&formula=1*(habitat%3E0)%2B1*(habitat%3E0)*((((cbr%20*%20(mar%20%2B%20apr%20%2B%20may))%3C(25.4*3)))%7C(((25.4*3)%3C%3D(cbr%20*%20(mar%20%2B%20apr%20%2B%20may)))%26((cbr%20*%20(mar%20%2B%20apr%20%2B%20may))%3C(25.4*7)))%7C(((25.4*7)%3C%3D(cbr%20*%20(mar%20%2B%20apr%20%2B%20may)))%26((cbr%20*%20(mar%20%2B%20apr%20%2B%20may))%3C(25.4*10)))%7C(((25.4*10)%3C%3D(cbr%20*%20(mar%20%2B%20apr%20%2B%20may)))%26((cbr%20*%20(mar%20%2B%20apr%20%2B%20may))%3C(25.4*13)))%7C(((25.4*13)%3C%3D(cbr%20*%20(mar%20%2B%20apr%20%2B%20may)))%26((cbr%20*%20(mar%20%2B%20apr%20%2B%20may))%3C(25.4*17)))%7C(((25.4*17)%3C%3D(cbr%20*%20(mar%20%2B%20apr%20%2B%20may)))%26((cbr%20*%20(mar%20%2B%20apr%20%2B%20may))%3C(25.4*20)))%7C(((25.4*20)%3C%3D(cbr%20*%20(mar%20%2B%20apr%20%2B%20may)))%26((cbr%20*%20(mar%20%2B%20apr%20%2B%20may))%3C(25.4*23)))%7C(((25.4*23)%3C%3D(cbr%20*%20(mar%20%2B%20apr%20%2B%20may)))%26((cbr%20*%20(mar%20%2B%20apr%20%2B%20may))%3C(25.4*27)))%7C(((cbr%20*%20(mar%20%2B%20apr%20%2B%20may))%3E(25.4*27))))'

In [23]:
# total spring precip 1990 = 375
# annual grassland = 20
url = 'http://greatbasin.geodesigntech.com/raster/export?'
url += 'bbox=-130.7373046875,33.063924198120645,-103.447265625,43.723474896114794'
url += '&zoom=6'
url += '&layers=habitat=41,mar=643,apr=481,may=447,cbr=647'
url += '&formula=1*(habitat%3E0)%2B1*(habitat%3E0)*((((cbr%20*%20(mar%20%2B%20apr%20%2B%20may))%3C(25.4*3)))%7C(((25.4*3)%3C%3D(cbr%20*%20(mar%20%2B%20apr%20%2B%20may)))%26((cbr%20*%20(mar%20%2B%20apr%20%2B%20may))%3C(25.4*7)))%7C(((25.4*7)%3C%3D(cbr%20*%20(mar%20%2B%20apr%20%2B%20may)))%26((cbr%20*%20(mar%20%2B%20apr%20%2B%20may))%3C(25.4*10)))%7C(((25.4*10)%3C%3D(cbr%20*%20(mar%20%2B%20apr%20%2B%20may)))%26((cbr%20*%20(mar%20%2B%20apr%20%2B%20may))%3C(25.4*13)))%7C(((25.4*13)%3C%3D(cbr%20*%20(mar%20%2B%20apr%20%2B%20may)))%26((cbr%20*%20(mar%20%2B%20apr%20%2B%20may))%3C(25.4*17)))%7C(((25.4*17)%3C%3D(cbr%20*%20(mar%20%2B%20apr%20%2B%20may)))%26((cbr%20*%20(mar%20%2B%20apr%20%2B%20may))%3C(25.4*20)))%7C(((25.4*20)%3C%3D(cbr%20*%20(mar%20%2B%20apr%20%2B%20may)))%26((cbr%20*%20(mar%20%2B%20apr%20%2B%20may))%3C(25.4*23)))%7C(((25.4*23)%3C%3D(cbr%20*%20(mar%20%2B%20apr%20%2B%20may)))%26((cbr%20*%20(mar%20%2B%20apr%20%2B%20may))%3C(25.4*27)))%7C(((cbr%20*%20(mar%20%2B%20apr%20%2B%20may))%3E(25.4*27))))'

In [24]:
# generated for comparison
url2 = 'http://greatbasin.geodesigntech.com/raster/export' 
# commas in bbox encoded
url2 += '&bbox=-130.7373046875%2C33.063924198120645%2C-103.447265625%2C43.723474896114794'
url2 += '&zoom=6'
# encoded internal = and ,
url2 += '&layers=habitat%3D41%2Cmar%3D643%2Capr%3D481%2Cmay%3D447%2Ccbr%3D647'
# encoded internal * (), where original encoded only + and space and /
url2 += '&formula=1%2A%28habitat%3E0%29%2B1%2A%28habitat%3E0%29%2A%28%28%28%28cbr+%2A+%28mar+%2B+apr+%2B+may%29%29%3C%2825.4%2A3%29%29%29%7C%28%28%2825.4%2A3%29%3C%3D%28cbr+%2A+%28mar+%2B+apr+%2B+may%29%29%29'

In [25]:
report_url = 'http://greatbasin.geodesigntech.com/gbreport/#{"layers"%3A{"driver"%3A684%2C"resource"%3A"720"}%2C"formula"%3A"(driver)*((resource))"%2C"legend"%3A[{"color"%3A"%23ffff00"%2C"code"%3A""%2C"expression"%3A"(x%20>%3D%200.1)%20and%20(x%20<%201)"%2C"name"%3A"0.1%20-%201.0"}%2C{"color"%3A"%23ff9900"%2C"code"%3A""%2C"expression"%3A"(x%20>%3D%201)%20and%20(x%20<%202)"%2C"name"%3A"1%20-%202"}%2C{"color"%3A"%23ff6600"%2C"code"%3A""%2C"expression"%3A"(x%20>%3D%202)%20and%20(%20x%20<%205)"%2C"name"%3A"2%20-%205"}%2C{"color"%3A"%23ff6600"%2C"code"%3A""%2C"expression"%3A"(x%20>%3D%205)%20and%20(x%20<%2010)"%2C"name"%3A"5%20-%2010"}%2C{"color"%3A"%23993300"%2C"code"%3A""%2C"expression"%3A"x%20>%2010"%2C"name"%3A">%2010"}]%2C"aggregationlayer"%3A5%2C"maxzoom"%3A8%2C"title"%3A"Mule%20Deer%202010%20Summer%20Habitat%20|%20Theobold%20Projected%20Urban%20Density%202030"%2C"model"%3A{"title"%3A"Mule%20Deer%202010%20Summer%20Habitat%20|%20Theobold%20Projected%20Urban%20Density%202030"}}'

In [26]:
durl = urllib.parse.unquote_plus(url)
durl
# note the internal & within the formula (which is reserved for param parsing in urls)

'http://greatbasin.geodesigntech.com/raster/export?bbox=-130.7373046875,33.063924198120645,-103.447265625,43.723474896114794&zoom=6&layers=habitat=41,mar=643,apr=481,may=447,cbr=647&formula=1*(habitat>0)+1*(habitat>0)*((((cbr * (mar + apr + may))<(25.4*3)))|(((25.4*3)<=(cbr * (mar + apr + may)))&((cbr * (mar + apr + may))<(25.4*7)))|(((25.4*7)<=(cbr * (mar + apr + may)))&((cbr * (mar + apr + may))<(25.4*10)))|(((25.4*10)<=(cbr * (mar + apr + may)))&((cbr * (mar + apr + may))<(25.4*13)))|(((25.4*13)<=(cbr * (mar + apr + may)))&((cbr * (mar + apr + may))<(25.4*17)))|(((25.4*17)<=(cbr * (mar + apr + may)))&((cbr * (mar + apr + may))<(25.4*20)))|(((25.4*20)<=(cbr * (mar + apr + may)))&((cbr * (mar + apr + may))<(25.4*23)))|(((25.4*23)<=(cbr * (mar + apr + may)))&((cbr * (mar + apr + may))<(25.4*27)))|(((cbr * (mar + apr + may))>(25.4*27))))'

In [27]:
sep = '?' # sometimes ?
gb_geotif_base_url = durl.split(sep, 1)[0]
print('Base URL is:')
print(gb_geotif_base_url)
print()
print('URL Parameter List is:')
gb_geotif_url_str = durl.split(sep, 1)[1]
print(gb_geotif_url_str)

Base URL is:
http://greatbasin.geodesigntech.com/raster/export

URL Parameter List is:
bbox=-130.7373046875,33.063924198120645,-103.447265625,43.723474896114794&zoom=6&layers=habitat=41,mar=643,apr=481,may=447,cbr=647&formula=1*(habitat>0)+1*(habitat>0)*((((cbr * (mar + apr + may))<(25.4*3)))|(((25.4*3)<=(cbr * (mar + apr + may)))&((cbr * (mar + apr + may))<(25.4*7)))|(((25.4*7)<=(cbr * (mar + apr + may)))&((cbr * (mar + apr + may))<(25.4*10)))|(((25.4*10)<=(cbr * (mar + apr + may)))&((cbr * (mar + apr + may))<(25.4*13)))|(((25.4*13)<=(cbr * (mar + apr + may)))&((cbr * (mar + apr + may))<(25.4*17)))|(((25.4*17)<=(cbr * (mar + apr + may)))&((cbr * (mar + apr + may))<(25.4*20)))|(((25.4*20)<=(cbr * (mar + apr + may)))&((cbr * (mar + apr + may))<(25.4*23)))|(((25.4*23)<=(cbr * (mar + apr + may)))&((cbr * (mar + apr + may))<(25.4*27)))|(((cbr * (mar + apr + may))>(25.4*27))))


In [28]:
param_list = gb_geotif_url_str.split('&') # how to avoid internal & in layers
param_list

['bbox=-130.7373046875,33.063924198120645,-103.447265625,43.723474896114794',
 'zoom=6',
 'layers=habitat=41,mar=643,apr=481,may=447,cbr=647',
 'formula=1*(habitat>0)+1*(habitat>0)*((((cbr * (mar + apr + may))<(25.4*3)))|(((25.4*3)<=(cbr * (mar + apr + may)))',
 '((cbr * (mar + apr + may))<(25.4*7)))|(((25.4*7)<=(cbr * (mar + apr + may)))',
 '((cbr * (mar + apr + may))<(25.4*10)))|(((25.4*10)<=(cbr * (mar + apr + may)))',
 '((cbr * (mar + apr + may))<(25.4*13)))|(((25.4*13)<=(cbr * (mar + apr + may)))',
 '((cbr * (mar + apr + may))<(25.4*17)))|(((25.4*17)<=(cbr * (mar + apr + may)))',
 '((cbr * (mar + apr + may))<(25.4*20)))|(((25.4*20)<=(cbr * (mar + apr + may)))',
 '((cbr * (mar + apr + may))<(25.4*23)))|(((25.4*23)<=(cbr * (mar + apr + may)))',
 '((cbr * (mar + apr + may))<(25.4*27)))|(((cbr * (mar + apr + may))>(25.4*27))))']

In [29]:
bbox = param_list[0].split('=',1)[1]
bbox

'-130.7373046875,33.063924198120645,-103.447265625,43.723474896114794'

In [30]:

zoom = param_list[1].split('=')[1]
zoom

'6'

In [31]:
# override zoom, for best scale reasonable on 4km grid climate rasters
zoom = 8

In [32]:
layers = param_list[2].split('=',1)[1]
layers

'habitat=41,mar=643,apr=481,may=447,cbr=647'

In [33]:
# this must go back to pre-&-split string, assuming formula is last in url
formula = gb_geotif_url_str.split('formula=')[1]
formula

'1*(habitat>0)+1*(habitat>0)*((((cbr * (mar + apr + may))<(25.4*3)))|(((25.4*3)<=(cbr * (mar + apr + may)))&((cbr * (mar + apr + may))<(25.4*7)))|(((25.4*7)<=(cbr * (mar + apr + may)))&((cbr * (mar + apr + may))<(25.4*10)))|(((25.4*10)<=(cbr * (mar + apr + may)))&((cbr * (mar + apr + may))<(25.4*13)))|(((25.4*13)<=(cbr * (mar + apr + may)))&((cbr * (mar + apr + may))<(25.4*17)))|(((25.4*17)<=(cbr * (mar + apr + may)))&((cbr * (mar + apr + may))<(25.4*20)))|(((25.4*20)<=(cbr * (mar + apr + may)))&((cbr * (mar + apr + may))<(25.4*23)))|(((25.4*23)<=(cbr * (mar + apr + may)))&((cbr * (mar + apr + may))<(25.4*27)))|(((cbr * (mar + apr + may))>(25.4*27))))'

In [34]:
# try hand-editing formula to desired results
formula = '(habitat>0)*(cbr * (mar + apr + may))'

In [35]:
# short for central basin and range
region_abr = 'CBR'

In [36]:
# 658 is intermountain big sagebrush and a huge layer
# 708 is tiny 4km res layer
#layers = 'habitat=708'
resource_layer_dict = get_raster_layer_dict(41)
resource_layer_dict

NameError: name 'get_raster_layer_dict' is not defined

In [37]:
resfile = resource_layer_dict['name'].replace(' ','_')
resfile

NameError: name 'resource_layer_dict' is not defined

In [38]:
res_name = 'Annual_Grasslands' # manually shortened...

In [ ]:
formula_name = 'Total_Spring_Precip_1990' # shortest manual


In [ ]:
filename = region_abr + '_' + res_name + "_" + formula_name
filename

What if I could replace and simplify formula...

(cbr * (mar + apr + may))

is the measure, which is only really understandable if you did into layer abbreviations.

So, in future, would be better if WMS-style time specifications were explicit, and perhaps also their "dimensions" with units.  So then precipitation in inches would be a dimension/unit, and dates/time ranges would be explicit modifiers, with explicit defaults (like most-recent time)

In [ ]:
layers

### Manualy request a single geoTIFF

In [ ]:
import urllib.parse

items = ['bbox', 'zoom', 'layers', 'formula']
url = ''
for i in items:
    param = '&{}='.format(str(i))
    element = str(eval(i))
    url += (param + urllib.parse.quote_plus(element))
url = gb_geotif_base_url + '?' + url
url

In [ ]:

path = filename + '.zip'
try:
    r = requests.get(url, headers=headers)
    r.raise_for_status()
except:
    print('Request failed')
# if we are ok, then save in chunks
if r.status_code == 200:
    with open(path, 'wb') as f:
        for chunk in r.iter_content(1024):
            f.write(chunk)
else:
    print('Request for data failed, error: {}'.format(responses[r.status_code]))
   

In [ ]:
!ls -l *.zip

In [ ]:
def extractGeoTIFF(url):
    durl = urllib.parse.unquote_plus(url)
    gbreport_base_url = durl.split('?', 1)[0]
    gbreport_obj = durl.split('?', 1)[1]
    report_dict = json.loads(gbreport_obj)
    resource, measure = report_dict['title'].split('|')
    print("Resource: {}".format(resource))
    print("Measure: {}".format(measure))

    layer_dict = report_dict['layers']
    print('Variable Layer_ID Layer_Name')
    for layer_abr, layer_id in layer_dict.items():
        layer_dict = get_raster_layer_dict(layer_id)
        print(layer_abr + ' ' + layer_id + layer_dict['name'])
        
    print("Formula: {}".format(report_dict['formula']))
    return(report_dict)

In [ ]:
extractGeoTIFF(url)

### Combination Loop

In [23]:
resources = lcd_df[lcd_df['type'] == 'resource']
resources.head(3)

,formula,layerid,layers,legend,legend_name,name,scenario,season,type,variable,year
2,x,401,x = 41,"[{'code': '', 'expression': '1', 'name': '1 - ...",Mask,Annual Grasslands,NaN,NaN,resource,NaN,NaN
3,x,414,x = 733,"[{'code': '5', 'expression': '1', 'name': 'Agr...",Desert Bighorn Sheep Habitat Types,California Desert Bighorn Sheep Distribution 2016,NaN,NaN,resource,NaN,NaN
7,x,413,x = 734,"[{'code': '5', 'expression': '1', 'name': 'Agr...",Desert Bighorn Sheep Habitat Types,Desert Bighorn Sheep Distribution 2016,NaN,NaN,resource,NaN,NaN


In [24]:
driverCount = len(scenarios['name'].unique())
print('Have {} drivers'.format(driverCount))

NameError: name 'scenarios' is not defined

In [41]:
resCount = len(resources['name'].unique())
print('Have {} resource maps'.format(resCount))

Have 18 resource maps


In [42]:
u =  'http://greatbasin.geodesigntech.com/gbreport/'
u += '#{"layers"%3A{"Dec60"%3A"620"%2C"Jan60"%3A"473"%2C"Feb60"%3A"584"%2C"CBR"%3A"647"%2C"resource"%3A"41"}%2C"formula"%3A"(((Dec60%2BJan60%2BFeb60)%2F3)*CBR)*((resource))"%2C"legend"%3A[{"color"%3A"%23ffeda0"%2C"code"%3A"1"%2C"expression"%3A"((1%20%2F%201.8)%20<%20x)%20%26%20(x%20<%20(2%20%2F%201.8))"%2C"name"%3A"01%20-%2002%20F"}%2C{"color"%3A"%23fed976"%2C"code"%3A"2"%2C"expression"%3A"((2%20%2F%201.8)%20<%20x)%20%26%20(x%20<%20(4%20%2F%201.8))"%2C"name"%3A"02%20-%2004%20F"}%2C{"color"%3A"%23feb24c"%2C"code"%3A"3"%2C"expression"%3A"((4%20%2F%201.8)%20<%20x)%20%26%20(x%20<%20(6%20%2F%201.8))"%2C"name"%3A"04%20-%2006%20F"}%2C{"color"%3A"%23fd8d3c"%2C"code"%3A"4"%2C"expression"%3A"((6%20%2F%201.8)%20<%20x)%20%26%20(x%20<%20(8%20%2F%201.8))"%2C"name"%3A"06%20-%2008%20F"}%2C{"color"%3A"%23fc4e2a"%2C"code"%3A"5"%2C"expression"%3A"((8%20%2F%201.8)%20<%20x)%20%26%20(x%20<%20(10%20%2F%201.8))"%2C"name"%3A"08%20-%2010%20F"}%2C{"color"%3A"%23e31a1c"%2C"code"%3A"6"%2C"expression"%3A"((10%20%2F%201.8)%20<%20x)%20%26%20(x%20<%20(12%20%2F%201.8))"%2C"name"%3A"10%20-%2012%20F"}%2C{"color"%3A"%23bd0026"%2C"code"%3A"7"%2C"expression"%3A"((12%20%2F%201.8)%20<%20x)%20%26%20(x%20<%20(14%20%2F%201.8))"%2C"name"%3A"12%20-%2014%20F"}%2C{"color"%3A"%23800026"%2C"code"%3A"8"%2C"expression"%3A"(14%20%2F%201.8)%20%20<%20x"%2C"name"%3A">%2014%20F"}]%2C"aggregationlayer"%3A5%2C"maxzoom"%3A8%2C"title"%3A"Annual%20Grasslands%20|%20RCP8.5%20Change%20in%20Max%20Temperature%202060%20Q4%20Winter"%2C"model"%3A{"title"%3A"Annual%20Grasslands%20|%20RCP8.5%20Change%20in%20Max%20Temperature%202060%20Q4%20Winter"}}'

In [43]:
url = 'http://greatbasin.geodesigntech.com/raster/export?&bbox=-130.7373046875%2C33.063924198120645%2C-103.447265625%2C43.723474896114794&zoom=6&layers=habitat%3D41%2Cmar%3D643%2Capr%3D481%2Cmay%3D447%2Ccbr%3D647&formula=%28habitat%3E0%29%2A%28cbr+%2A+%28mar+%2B+apr+%2B+may%29%29'

In [65]:
def exportAllExposures():
    bbox_param = '&bbox=-130.95703125,33.063924198120645,-103.22753906249999,43.723474896114794'
    zoom_param = '&zoom=8' # for climate change res

    limit = 0
    count = 0
    overwrite = False #default is skip if previously written
    
    comboCount = resCount * driverCount
    print("Starting exposure generation - {} resources, {} drivers, total combinations = {}".format(resCount, driverCount, comboCount))

    for res in resources.itertuples():
        if limit > 0 and count < limit:
            print('{} - Resource: {} - {}'.format(count, res.layerid, res.name))
        legend_entries_list = res.legend
        #for e in legend_entries_list:
        #    print(e['name'])
        res_name = (res.name).replace(' ','_')
        res_layer = res.layers.split('=')[1].strip() # 'x = 123' -> 123
        
        for driver in scenarios.itertuples():
            if limit > 0 and count >= limit:
                continue #or exit?
            count += 1
            driver_name = (driver.name).replace(' ','_')
            filename = 'cbr_{}_{}'.format(res_name, driver_name)  
            path = filename + '.zip'

            # check if file already exists and skip unless override flag is on
            if os.path.exists(path):
                print('Skipping previously existing file {}'.format(path))
                continue
                
            driver_id = driver.layerid
            driver_layers = (driver.layers).replace(' ','') # just despace should be sufficient
            #driver_formula = urllib.parse.quote_plus(driver.formula) # standard encode?..
            # seems too harsh, but go with just known problems to retain parentheses?
            driver_formula = (driver.formula).replace(' ','').replace('+','%2B').replace('/','%2F')
            print("     Driver: {} - {}".format(driver.layerid, driver.name))
            # driver rasterids are in its layers
            layers = "resource={},{}".format(res_layer,driver_layers)
            layers_param = "&layers={}".format(layers)
            # need to pick up driver formula as subformula here
            formula = '(resource>0)*({})'.format(driver_formula)
            formula_param = "&formula={}".format(formula)
            url = gb_geotif_base_url + '?' + bbox_param + zoom_param + layers_param + formula_param
            print('url to request geotiff is:')
            print(url)
            try:
                r = requests.get(url, headers=headers)
                r.raise_for_status()
            except:
                print('Problem with geotiff export: {}.\n'.format(responses[r.status_code]))
                continue # keep looping on error
            print('Writing file {}'.format(path))
            with open(path, 'wb') as f:
                for chunk in r.iter_content(4096):
                    f.write(chunk)

        print()

    print("Done")

In [ ]:
exportAllExposures()

Starting exposure generation - 18 resources, 54 drivers, total combinations = 972
Skipping previously existing file cbr_Annual_Grasslands_Fore-Sce_A1b_Land_Cover_Change_2010-2050.zip
Skipping previously existing file cbr_Annual_Grasslands_Fore-Sce_A1b_Urbanization_2010-2040.zip
Skipping previously existing file cbr_Annual_Grasslands_Fore-Sce_A2_Land_Cover_Change_2050.zip
Skipping previously existing file cbr_Annual_Grasslands_Fore-sce_Scenario_A1B_2050.zip
Skipping previously existing file cbr_Annual_Grasslands_Fore-sce_Scenario_A2_2050.zip
Skipping previously existing file cbr_Annual_Grasslands_Fore-sce_Scenario_B1_2050.zip
Skipping previously existing file cbr_Annual_Grasslands_Great_Basin_Pinyon_Juniper_Woodland_2010.zip
Skipping previously existing file cbr_Annual_Grasslands_Greater_Sage-grouse.zip
Skipping previously existing file cbr_Annual_Grasslands_Greater_Sage_Grouse_2050_Bioclimate.zip
     Driver: 375 - Historical 1990 Q1 Spring Total Precipitation
url to request geotiff is

Writing file cbr_Annual_Grasslands_RCP4.5_Change_in_Average_Monthly_Precipitation_2060_Q4_Winter.zip
Skipping previously existing file cbr_Annual_Grasslands_RCP4.5_Change_in_Average_Monthly_Precipitation_2060_Q4_Winter.zip
     Driver: 366 - RCP4.5 Change in Max Temperature 2060 Q1 Spring
url to request geotiff is:
http://greatbasin.geodesigntech.com/raster/export?&bbox=-130.95703125,33.063924198120645,-103.22753906249999,43.723474896114794&zoom=8&layers=resource=41,Mar60=588,Apr60=587,May60=451,CBR=647&formula=(resource>0)*(((Mar60%2BApr60%2BMay60)%2F3)*CBR)
Writing file cbr_Annual_Grasslands_RCP4.5_Change_in_Max_Temperature_2060_Q1_Spring.zip
     Driver: 378 - RCP4.5 Change in Max Temperature 2060 Q3 Fall
url to request geotiff is:
http://greatbasin.geodesigntech.com/raster/export?&bbox=-130.95703125,33.063924198120645,-103.22753906249999,43.723474896114794&zoom=8&layers=resource=41,sep=600,oct=510,nov=608,cbr=647&formula=(resource>0)*(((sep%2Boct%2Bnov)%2F3)*cbr)
Writing file cbr_A

Writing file cbr_California_Desert_Bighorn_Sheep_Distribution_2016_Historical_Max_Air_Temperature_1990_Q1_Spring.zip
     Driver: 362 - Historical Max Air Temperature 1990 Q2 Summer
url to request geotiff is:
http://greatbasin.geodesigntech.com/raster/export?&bbox=-130.95703125,33.063924198120645,-103.22753906249999,43.723474896114794&zoom=8&layers=resource=733,June=622,July=492,Aug=543&formula=(resource>0)*((June%2BJuly%2BAug)%2F3)
Writing file cbr_California_Desert_Bighorn_Sheep_Distribution_2016_Historical_Max_Air_Temperature_1990_Q2_Summer.zip
     Driver: 363 - Historical Max Air Temperature 1990 Q3 Fall
url to request geotiff is:
http://greatbasin.geodesigntech.com/raster/export?&bbox=-130.95703125,33.063924198120645,-103.22753906249999,43.723474896114794&zoom=8&layers=resource=733,Sept=645,Oct=461,Nov=518&formula=(resource>0)*((Sept%2BOct%2BNov)%2F3)
Writing file cbr_California_Desert_Bighorn_Sheep_Distribution_2016_Historical_Max_Air_Temperature_1990_Q3_Fall.zip
     Driver: 36

Problem with geotiff export: Bad Request.

     Driver: 431 - RCP8.5 Change in Average Monthly Precipitation 2060 Q1 Summer
url to request geotiff is:
http://greatbasin.geodesigntech.com/raster/export?&bbox=-130.95703125,33.063924198120645,-103.22753906249999,43.723474896114794&zoom=8&layers=resource=733,jun90=450,jul90=611,aug90=505,jun=430,jul=564,aug=444&formula=(resource>0)*((((jun%2Bjul%2Baug)%2F3-((mar90%2Bapr90%2Bmay90)%2F3))*cbr)
Problem with geotiff export: Bad Request.

     Driver: 432 - RCP8.5 Change in Average Monthly Precipitation 2060 Q3 Fall
url to request geotiff is:
http://greatbasin.geodesigntech.com/raster/export?&bbox=-130.95703125,33.063924198120645,-103.22753906249999,43.723474896114794&zoom=8&layers=resource=733,x=None&formula=(resource>0)*(x)
Problem with geotiff export: Bad Request.

     Driver: 387 - RCP8.5 Change in Average Monthly Precipitation 2060 Q4 Winter
url to request geotiff is:
http://greatbasin.geodesigntech.com/raster/export?&bbox=-130.95703125,3

Writing file cbr_Desert_Bighorn_Sheep_Distribution_2016_RCP4.5_Change_in_Average_Monthly_Precipitation_2060_Q1_Spring.zip
     Driver: 370 - RCP4.5 Change in Average Monthly Precipitation 2060 Q2 Summer
url to request geotiff is:
http://greatbasin.geodesigntech.com/raster/export?&bbox=-130.95703125,33.063924198120645,-103.22753906249999,43.723474896114794&zoom=8&layers=resource=734,Jun60=430,Jul60=564,Aug60=444,CBR=647&formula=(resource>0)*(((Jun60%2BJul60%2BAug60)%2F3)*CBR)
Writing file cbr_Desert_Bighorn_Sheep_Distribution_2016_RCP4.5_Change_in_Average_Monthly_Precipitation_2060_Q2_Summer.zip
     Driver: 382 - RCP4.5 Change in Average Monthly Precipitation 2060 Q3 Fall
url to request geotiff is:
http://greatbasin.geodesigntech.com/raster/export?&bbox=-130.95703125,33.063924198120645,-103.22753906249999,43.723474896114794&zoom=8&layers=resource=734,sep=433,oct=642,nov=442,cbr=647&formula=(resource>0)*(((sep%2Boct%2Bnov)%2F3)*cbr)
Writing file cbr_Desert_Bighorn_Sheep_Distribution_201

...

Writing file cbr_Mule_Deer_2050_Year_Round_Bioclimate_Historical_Max_Air_Temperature_1990_Q4_Winter.zip
Skipping previously existing file cbr_Mule_Deer_2050_Year_Round_Bioclimate_Historical_Precipitation_1990_M01_January.zip
Skipping previously existing file cbr_Mule_Deer_2050_Year_Round_Bioclimate_Historical_Precipitation_1990_M02_February.zip
Skipping previously existing file cbr_Mule_Deer_2050_Year_Round_Bioclimate_Historical_Precipitation_1990_M03_March.zip
Skipping previously existing file cbr_Mule_Deer_2050_Year_Round_Bioclimate_Historical_Precipitation_1990_M04_April.zip
Skipping previously existing file cbr_Mule_Deer_2050_Year_Round_Bioclimate_Historical_Precipitation_1990_M05_May.zip
Skipping previously existing file cbr_Mule_Deer_2050_Year_Round_Bioclimate_Historical_Precipitation_1990_M06_June.zip
Skipping previously existing file cbr_Mule_Deer_2050_Year_Round_Bioclimate_Historical_Precipitation_1990_M07_July.zip
Skipping previously existing file cbr_Mule_Deer_2050_Year_Roun

Writing file cbr_Mule_Deer_2050_Year_Round_Bioclimate_RCP8.5_Change_in_Max_Temperature_2060_Q1_Spring.zip
     Driver: 372 - RCP8.5 Change in Max Temperature 2060 Q4 Winter
url to request geotiff is:
http://greatbasin.geodesigntech.com/raster/export?&bbox=-130.95703125,33.063924198120645,-103.22753906249999,43.723474896114794&zoom=8&layers=resource=702,Dec60=620,Jan60=473,Feb60=584,CBR=647&formula=(resource>0)*(((Dec60%2BJan60%2BFeb60)%2F3)*CBR)
Writing file cbr_Mule_Deer_2050_Year_Round_Bioclimate_RCP8.5_Change_in_Max_Temperature_2060_Q4_Winter.zip
Skipping previously existing file cbr_Mule_Deer_2050_Year_Round_Bioclimate_Sonora_Mojave_Semi-Desert_Chaparral.zip

Skipping previously existing file cbr_Pygmy_Rabbit_2050_Bioclimate_Fore-Sce_A1b_Land_Cover_Change_2010-2050.zip
Skipping previously existing file cbr_Pygmy_Rabbit_2050_Bioclimate_Fore-Sce_A1b_Urbanization_2010-2040.zip
Skipping previously existing file cbr_Pygmy_Rabbit_2050_Bioclimate_Fore-Sce_A2_Land_Cover_Change_2050.zip
Ski

Writing file cbr_Pygmy_Rabbit_2050_Bioclimate_RCP4.5_Change_in_Average_Monthly_Precipitation_2060_Q1_Spring.zip
     Driver: 370 - RCP4.5 Change in Average Monthly Precipitation 2060 Q2 Summer
url to request geotiff is:
http://greatbasin.geodesigntech.com/raster/export?&bbox=-130.95703125,33.063924198120645,-103.22753906249999,43.723474896114794&zoom=8&layers=resource=724,Jun60=430,Jul60=564,Aug60=444,CBR=647&formula=(resource>0)*(((Jun60%2BJul60%2BAug60)%2F3)*CBR)
Writing file cbr_Pygmy_Rabbit_2050_Bioclimate_RCP4.5_Change_in_Average_Monthly_Precipitation_2060_Q2_Summer.zip
     Driver: 382 - RCP4.5 Change in Average Monthly Precipitation 2060 Q3 Fall
url to request geotiff is:
http://greatbasin.geodesigntech.com/raster/export?&bbox=-130.95703125,33.063924198120645,-103.22753906249999,43.723474896114794&zoom=8&layers=resource=724,sep=433,oct=642,nov=442,cbr=647&formula=(resource>0)*(((sep%2Boct%2Bnov)%2F3)*cbr)
Writing file cbr_Pygmy_Rabbit_2050_Bioclimate_RCP4.5_Change_in_Average_Mon

Writing file cbr_Pygmy_Rabbit_Future_Bioclimate_2050_Historical_1990_Q3_Fall_Total_Precipitation.zip
     Driver: 374 - Historical 1990 Q4 Winter Total Precipitation
url to request geotiff is:
http://greatbasin.geodesigntech.com/raster/export?&bbox=-130.95703125,33.063924198120645,-103.22753906249999,43.723474896114794&zoom=8&layers=resource=724,dec=476,jan=506,feb=546,cbr=647&formula=(resource>0)*(cbr*(dec%2Bjan%2Bfeb))
Writing file cbr_Pygmy_Rabbit_Future_Bioclimate_2050_Historical_1990_Q4_Winter_Total_Precipitation.zip
     Driver: 373 - Historical 1990 Total Annual Precipitation
url to request geotiff is:
http://greatbasin.geodesigntech.com/raster/export?&bbox=-130.95703125,33.063924198120645,-103.22753906249999,43.723474896114794&zoom=8&layers=resource=724,jan=506,feb=546,mar=643,apr=481,may=447,jun=450,jul=611,aug=505,sep=532,oct=440,nov=628,dec=476,cbr=647&formula=(resource>0)*((jan%2Bfeb%2Bmar%2Bapr%2Bmay%2Bjun%2Bjul%2Baug%2Bsep%2Boct%2Bnov%2Bdec)*cbr)
Writing file cbr_Pygmy_Ra

Writing file cbr_Pygmy_Rabbit_Future_Bioclimate_2050_RCP4.5_Change_in_Max_Temperature_2060_Q4_Winter.zip
     Driver: 369 - RCP4.5 Change in Precipitation 2060 Q2 Summer
url to request geotiff is:
http://greatbasin.geodesigntech.com/raster/export?&bbox=-130.95703125,33.063924198120645,-103.22753906249999,43.723474896114794&zoom=8&layers=resource=724,Jun60=634,Jul60=539,Aug60=455,CBR=647&formula=(resource>0)*(((Jun60%2BJul60%2BAug60)%2F3)*CBR)
Writing file cbr_Pygmy_Rabbit_Future_Bioclimate_2050_RCP4.5_Change_in_Precipitation_2060_Q2_Summer.zip
     Driver: 385 - RCP8.5 Change in Average Monthly Precipitation 2060 Q1 Spring
url to request geotiff is:
http://greatbasin.geodesigntech.com/raster/export?&bbox=-130.95703125,33.063924198120645,-103.22753906249999,43.723474896114794&zoom=8&layers=resource=724,cbr=647,mar60=563,apr60=537,may60=627,mar90=643,apr90=481,may90=447&formula=(resource>0)*((((mar%2Bapr%2Bmay)%2F3-((mar90%2Bapr90%2Bmay90)%2F3))*cbr)
Problem with geotiff export: Bad Requ

#### compare interface-generated url to machine generated above

In [ ]:
m_url = 'http://greatbasin.geodesigntech.com/raster/export?
&bbox=-130.95703125,33.063924198120645,-103.22753906249999,43.723474896114794
&zoom=8
# is resource the landcover layer id instead of the grid id?
&layers=resource=401,A1b2050=773,A2_2010=779
&formula=(resource>0)*((A1b2050-A2_2010)<>0)'

In [ ]:
http://greatbasin.geodesigntech.com/raster/export?&bbox=-130.95703125,33.063924198120645,-103.22753906249999,43.723474896114794&zoom=8&layers=resource=41,A1b2050=773,A2_2010=779&formula=(resource>0)*((A1b2050-A2_2010)<>0)

In [ ]:
i_url = 'http://greatbasin.geodesigntech.com/raster/export?
bbox=-127.4853515625,33.119150226768866,-100.1953125,43.77109381775648
&zoom=6
&layers=habitat=41,x=670
&formula=1*(habitat%3E0)%2B1*(habitat%3E0)*((x%3D%3D1)%7C(x%3D%3D2)%7C(x%3D%3D3)%7C(x%3D%3D4)%7C(x%3D%3D5)%7C(x%3D%3D6)%7C(x%3D%3D7)%7C(x%3D%3D8)%7C(x%3D%3D9)%7C(x%3D%3D10)%7C(x%3D%3D11)%7C(x%3D%3D12)%7C(x%3D%3D13)%7C(x%3D%3D14))'

In [ ]:
i_url = 'http://greatbasin.geodesigntech.com/raster/export?'
i_url += 'bbox=-127.4853515625,33.119150226768866,-100.1953125,43.77109381775648'
i_url += '&zoom=6&layers=habitat=41,x=506'
i_url += '&formula=1*(habitat%3E0)%2B1*(habitat%3E0)*(((x%3C(-25.4*2.3)))%7C(((25.4*-2.3)%3C%3Dx)%26(x%3C(25.4*-1.7)))%7C(((25.4*-1.7)%3C%3Dx)%26(x%3C(25.4*-1)))%7C(((25.4*-1)%3C%3Dx)%26(x%3C(25.4*-0.3)))%7C(((25.4*0.3)%3C%3Dx)%26(x%3C(25.4*1)))%7C(((25.4*1)%3C%3Dx)%26(x%3C(25.4*1.7)))%7C(((25.4*1.7)%3C%3Dx)%26(x%3C(25.4*2.3)))%7C((x%3E(25.4*2.3))))'

In [ ]:
# driver = 506 above and 220 below
# habitat = 41 above and 401 below
# 401 confirmed landcover annual grasslands
# 41 not a valid landcover, but is valid rasterid 
# 506 is raster layer for jan precip

# so the problem is that we are constructing dictionary based on landcover
# but not underlying grids ids

In [ ]:
m_url = 'http://greatbasin.geodesigntech.com/raster/export?'
m_url += '&bbox=-130.95703125,33.063924198120645,-103.22753906249999,43.723474896114794'
m_url += '&zoom=8&layers=resource=401,driver=220,cbr=647'
m_url += '&formula=(resource>0)*driver*cbr'

In [ ]:
resources[resources['layerid'] == 401]

In [ ]:
scenarios[scenarios['layerid'] == 220]

In [ ]:
# so, for both subtypes, we are not getting layers correctly defined

In [ ]:
server_url = ''
formula_url = '"formula":"(driver>0)*(resource)"'

for res in resources.iterrows():
    #print(res)
    res_name = resources.iloc[]
    for dr in drivers:
        driver_name = dr['name']
        print('Working on resource {} and driver {}'.format(res_name, driver_name))
        
        layers = ''
        items = ['bbox', 'zoom', 'layers', 'formula']
        url = ''
        for i in items:
            param = '&{}='.format(str(i))
            element = str(eval(i))
            url += (param + element)
        url = gb_geotif_base_url + url
        print('Url to request geotiff is:')
        print(url)

### Generate a Data Index

In [76]:
import pandas as pd
def createExposureIndex():
    rows_list = []
    count = 0

    for res in resources.itertuples():
        res_name = (res.name).replace(' ','_')
        
        for driver in scenarios.itertuples():
            count += 1
            driver_name = (driver.name).replace(' ','_')
            filename = 'cbr_{}_{}'.format(res_name, driver_name)  
            path = filename + '.zip'

            fdict = {}
            fdict.update({'Fileno': count, 'Resource': res.name, 'Driver': driver.name, 'Filename': path})

            rows_list.append(fdict)

    df = pd.DataFrame(rows_list)               

    print("Done")
    return(df)

In [77]:
df = createExposureIndex()
df.head()

Done


,Driver,Filename,Fileno,Resource
0,Fore-Sce A1b Land Cover Change 2010-2050,cbr_Annual_Grasslands_Fore-Sce_A1b_Land_Cover_...,1,Annual Grasslands
1,Fore-Sce A1b Urbanization 2010-2040,cbr_Annual_Grasslands_Fore-Sce_A1b_Urbanizatio...,2,Annual Grasslands
2,Fore-Sce A2 Land Cover Change 2050,cbr_Annual_Grasslands_Fore-Sce_A2_Land_Cover_C...,3,Annual Grasslands
3,Fore-sce Scenario A1B 2050,cbr_Annual_Grasslands_Fore-sce_Scenario_A1B_20...,4,Annual Grasslands
4,Fore-sce Scenario A2 2050,cbr_Annual_Grasslands_Fore-sce_Scenario_A2_205...,5,Annual Grasslands


In [78]:
#df = df[['mean', 4,3,2,1]]
df = df[['Fileno','Resource','Driver','Filename']]

In [80]:
df.to_csv('file_list.csv', index=False)

In [81]:
!head file_list.csv

Fileno,Resource,Driver,Filename
1,Annual Grasslands,Fore-Sce A1b Land Cover Change 2010-2050,cbr_Annual_Grasslands_Fore-Sce_A1b_Land_Cover_Change_2010-2050.zip
2,Annual Grasslands,Fore-Sce A1b Urbanization 2010-2040,cbr_Annual_Grasslands_Fore-Sce_A1b_Urbanization_2010-2040.zip
3,Annual Grasslands,Fore-Sce A2 Land Cover Change 2050,cbr_Annual_Grasslands_Fore-Sce_A2_Land_Cover_Change_2050.zip
4,Annual Grasslands,Fore-sce Scenario A1B 2050,cbr_Annual_Grasslands_Fore-sce_Scenario_A1B_2050.zip
5,Annual Grasslands,Fore-sce Scenario A2 2050,cbr_Annual_Grasslands_Fore-sce_Scenario_A2_2050.zip
6,Annual Grasslands,Fore-sce Scenario B1 2050,cbr_Annual_Grasslands_Fore-sce_Scenario_B1_2050.zip
7,Annual Grasslands,Great Basin Pinyon Juniper Woodland 2010,cbr_Annual_Grasslands_Great_Basin_Pinyon_Juniper_Woodland_2010.zip
8,Annual Grasslands,Greater Sage-grouse,cbr_Annual_Grasslands_Greater_Sage-grouse.zip
9,Annual Grasslands,Greater Sage Grouse 2050 Bioclimate,cbr_Annual_Grasslands_Greater_Sage_Grous

In [83]:
df.to_html('file_list.html')

In [84]:
style = '<style>.right_aligned_df td { text-align: right; }</style>'
HTML(style + df.to_html(formatters=frmt, classes='right_aligned_df'))


NameError: name 'HTML' is not defined